<a href="https://colab.research.google.com/github/ktam069/Google_Colab_Audiovisual/blob/master/AV_Layers_on_CPU_Audio_Only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive to access npy dataset of audio input for training/testing

In [22]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Main program - Part IV Project 80 Basic Version

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, TimeDistributed, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
import os
# import sys
import glob
from datetime import datetime

In [26]:
# Main program - Part IV Project 80 Basic Version

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

import tensorflow as tf
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Conv2D, LSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, Flatten, TimeDistributed, Reshape
from keras.callbacks import ModelCheckpoint
from keras.layers import concatenate
from keras.layers import Lambda

import os
# import sys
import glob
from datetime import datetime

# ===== Settings =====

# Display options
PRINT_DATA = False
DISPLAY_GRAPHS = False

# Data transformation/processing
NORMALISE_DATA = False
POWER_ENCODE = True

# Use float32 format for dataset ndarrays
USE_FLOAT32 = True

# Range of data to use for training (excludes END_ID)
START_ID = 0
END_ID = 21

# Sampling rate
SAMPLING_RATE = 16000

# Filepaths to the locations for input audio-visual files
# path_to_data = "./data/"
path_to_data_audio = "./data/audio/audio_train/"
path_to_data_video = "./data/video/face_input/"
# audio_data_name = "trim_audio_train%d.wav"

# Filepaths to the locations for saved data and models
path_to_models = "./saved_models/"
path_to_saved_datasets = "./dataset_npy/"
path_to_outputs = "./output_wavs/"

# ====================

'''def convolution_model(num_speakers=2):'''

num_speakers=2

# == Audio convolution layers ==

model = Sequential()

# # Implicit input layer
# inputs = Input(shape=(298, 257, 2))
# model.add(inputs)

# Convolution layers
conv1 = Conv2D(96, kernel_size=(1,7), padding='same', dilation_rate=(1,1), input_shape=(298, 257, 2), name="input_layer")
model.add(conv1)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv2 = Conv2D(96, kernel_size=(7,1), padding='same', dilation_rate=(1,1))
model.add(conv2)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv3 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(1,1))
model.add(conv3)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv4 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(2,1))
model.add(conv4)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv5 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(4,1))
model.add(conv5)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv6 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(8,1))
model.add(conv6)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv7 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(16,1))
model.add(conv7)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv8 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(32,1))
model.add(conv8)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv9 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(1,1))
model.add(conv9)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv10 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(2,2))
model.add(conv10)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv11 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(4,4))
model.add(conv11)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv12 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(8,8))
model.add(conv12)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv13 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(16,16))
model.add(conv13)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv14 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(32,32))
model.add(conv14)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv15 = Conv2D(8, kernel_size=(1,1), padding='same', dilation_rate=(1,1))
model.add(conv15)
model.add(BatchNormalization())
model.add(Activation("relu"))

# == AV fused neural network ==

# AV fusion step(s)
model.add(TimeDistributed(Flatten()))

# BLSTM
new_matrix_length = 400
model.add(Bidirectional(LSTM(new_matrix_length//2, return_sequences=True, input_shape=(298, 257*8))))

# Fully connected layers
model.add(Dense(600, activation="relu"))
model.add(Dense(600, activation="relu"))
model.add(Dense(600, activation="relu"))

# Output layer (i.e. complex masks)
# outputs = Dense(257*2*num_speakers, activation="relu")
outputs = Dense(257*2*num_speakers, activation="sigmoid")				# TODO: check if this is more correct (based on the paper)
model.add(outputs)
outputs_complex_masks = Reshape((298, 257, 2, num_speakers), name="output_layer")
model.add(outputs_complex_masks)

# Print the output shapes of each model layer
for layer in model.layers:
  name = layer.get_config()["name"]
  if "batch_normal" in name or "activation" in name:
    continue
  print(layer.output_shape, "\t", name)

# Alternatively, print the default keras model summary
model.summary()

# Compile the model before training
# model.compile(optimizer='adam', loss='mse')
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


(None, 298, 257, 96) 	 input_layer
(None, 298, 257, 96) 	 conv2d_29
(None, 298, 257, 96) 	 conv2d_30
(None, 298, 257, 96) 	 conv2d_31
(None, 298, 257, 96) 	 conv2d_32
(None, 298, 257, 96) 	 conv2d_33
(None, 298, 257, 96) 	 conv2d_34
(None, 298, 257, 96) 	 conv2d_35
(None, 298, 257, 96) 	 conv2d_36
(None, 298, 257, 96) 	 conv2d_37
(None, 298, 257, 96) 	 conv2d_38
(None, 298, 257, 96) 	 conv2d_39
(None, 298, 257, 96) 	 conv2d_40
(None, 298, 257, 96) 	 conv2d_41
(None, 298, 257, 8) 	 conv2d_42
(None, 298, 2056) 	 time_distributed_3
(None, 298, 400) 	 bidirectional_3
(None, 298, 600) 	 dense_9
(None, 298, 600) 	 dense_10
(None, 298, 600) 	 dense_11
(None, 298, 1028) 	 dense_12
(None, 298, 257, 2, 2) 	 output_layer
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Conv2D)         (None, 298, 257, 96)      1440      
____________________________________________________________

In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/dataset_train_0-20.npy' "dataset_train_0-20.npy"

In [28]:
!ls

ablation_basic_times_ao.csv  dataset_train_0-20.npy  gdrive  sample_data


In [0]:
dataset_train = np.load("dataset_train_0-20.npy")

In [0]:
# Split dataset into inputs and ground truths
x_train = dataset_train[:,:,:,:,0 ]
y_train = dataset_train[:,:,:,:,1:]


x_visual = np.random.rand(len(x_train), 75, 1, 1024)

In [0]:
import time

In [32]:
times = []
num_tests = 10

for i in range(num_tests):
	'''Predict data'''
	start_time = time.time()
	model.predict(x_train[i:i+1])
	#model.predict([x_train[i:i+1], x_visual[i:i+1], x_visual[i:i+1]])
	end_time = time.time()
	runtime = end_time-start_time
	times.append(runtime)
	print("Time taken for run %d:"%(i+1), runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Time taken for run 1: 10.601673364639282
Time taken for run 2: 9.145011901855469
Time taken for run 3: 9.067847490310669
Time taken for run 4: 9.053540706634521
Time taken for run 5: 9.028331756591797
Time taken for run 6: 9.036173105239868
Time taken for run 7: 9.069024324417114
Time taken for run 8: 9.054639101028442
Time taken for run 9: 9.067076206207275
Time taken for run 10: 9.04939889907837
Number of test: 10
Test times: [10.601673364639282, 9.145011901855469, 9.067847490310669, 9.053540706634521, 9.028331756591797, 9.036173105239868, 9.069024324417114, 9.054639101028442, 9.067076206207275, 9.04939889907837]
Average running time: 9.21727168560028


In [0]:
fname = "ablation_basic_times_ao.csv"
np.savetxt(fname, times, delimiter=",")

In [0]:
new_dir = "/content/gdrive/My Drive/Part IV Project/Colab Testing Structure/AV_basic_ablation_output/"
if not os.path.isdir(new_dir):
  os.mkdir(new_dir)

In [0]:
!cp $fname "{new_dir}{fname}"

In [36]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Conv2D)         (None, 298, 257, 96)      1440      
_________________________________________________________________
batch_normalization_31 (Batc (None, 298, 257, 96)      384       
_________________________________________________________________
activation_31 (Activation)   (None, 298, 257, 96)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 298, 257, 96)      64608     
_________________________________________________________________
batch_normalization_32 (Batc (None, 298, 257, 96)      384       
_________________________________________________________________
activation_32 (Activation)   (None, 298, 257, 96)      0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 298, 257, 96)     